# Work in progress

In [1]:
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("Scala Machine Learning Clustering").getOrCreate()

val file = "/home/jovyan/Resources/tx.csv"
val text = spark.sparkContext.textFile(file)

val txns = text.map(s => s.split(",")).map(a => a(2).toFloat)

// get the amounts for clustering training, the amounts need to be vectorized
val amnts = txns.map(v => Vectors.dense(v)).cache

val clusterCount = 10
val iterationCount = 20

// training the model
val model = KMeans.train(amnts, clusterCount, iterationCount)

println("Cluster centers")

// zipping with index to get the cluster label
model.clusterCenters.zipWithIndex.map(c => (c._2, c._1(0))).foreach(println(_))

// generate the amounts for clustering: +- 5*10^i, i=0..3
val vals = sc.parallelize(0 to 3).map(i => Math.pow(10, i)).flatMap(x => Array(-5*x, 5*x)).map(v => Vectors.dense(v))

// predict the clusters for the amounts
val clsd = vals.map(s => (s(0), model.predict(s)))

println("Predictions")

clsd.collect.foreach(println(_))


Cluster centers
(0,137.71837801546664)
(1,-15.051251378631923)
(2,2209.030029296875)
(3,825.0619995117188)
(4,-362.9444410536024)
(5,1440.239990234375)
(6,-819.5774993896484)
(7,-191.42642865862163)
(8,389.8246154785156)
(9,-72.59154530144556)
Predictions
(-5.0,1)
(5.0,1)
(-50.0,9)
(50.0,1)
(-500.0,4)
(500.0,8)
(-5000.0,6)
(5000.0,2)


spark = org.apache.spark.sql.SparkSession@15bbf8d6
file = /home/jovyan/Resources/tx.csv
text = /home/jovyan/Resources/tx.csv MapPartitionsRDD[1] at textFile at <console>:37
txns = MapPartitionsRDD[3] at map at <console>:39
amnts = MapPartitionsRDD[4] at map at <console>:42
clusterCount = 10
iterationCount = 20
model = org.apache.spark.mllib.clustering.KMeansModel@2bb83aaf


vals: org.apache.spark.rdd.RDD[org.apache....


org.apache.spark.mllib.clustering.KMeansModel@2bb83aaf

## Challenge

#### FInd the "optimal" number of clusters for transaction amounts

In [2]:
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("Scala Machine Learning Clustering").getOrCreate()

val file = "/home/jovyan/Resources/tx.csv"
val text = spark.sparkContext.textFile(file)

val txns = text.map(s => s.split(",")).map(a => a(2).toFloat)

// get the amounts for clustering training, the amounts need to be vectorized
val amnts = txns.map(v => Vectors.dense(v)).cache

val iterationCount = 20

println("Within Set Sum of Squared Errors")
for (clusterCount <- 1 to 20) {
  val model = KMeans.train(amnts, clusterCount, iterationCount)

  val wssse = model.computeCost(amnts)
  println(clusterCount + "\t" + Math.round(wssse))
}


Within Set Sum of Squared Errors
1	22058151
2	10992591
3	7954986
4	6712829
5	3361231
6	2472643
7	1898504
8	1416450
9	1150353
10	766466
11	834030
12	509116
13	460154
14	353169
15	331615
16	285519
17	236213
18	194126
19	195423
20	161736


spark = org.apache.spark.sql.SparkSession@15bbf8d6
file = /home/jovyan/Resources/tx.csv
text = /home/jovyan/Resources/tx.csv MapPartitionsRDD[64] at textFile at <console>:45
txns = MapPartitionsRDD[66] at map at <console>:47
amnts = MapPartitionsRDD[67] at map at <console>:50
iterationCount = 20


20

Look for an "elbow" in WSSSE graph. We think the optimal number of clusters is 9.